# Beyond Linear: Going Hierarchical
All nice and well but linear models with only one predictor can be a bit boring, right? Also, most people know that size is not the only factor determining the rental price. If you've lived in Berlin for a while, you know that certain areas are much more expensive than others.

Unfortunately, this data set doesn't contain the coordinates for each flat nor the exact address. But for each flat we have a ZIP code (the PLZ).
We will now extend our model to incorporate the location by using the ZIP code. We do so by training one linear model per ZIP code.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pymc3 as pm
import theano

import sys
sys.path.append('../src/')
from utils import standardize_area, destandardize_area

berlin = pd.read_csv("../data/berlin.csv", index_col=0, dtype={"geo_plz":str})

In [ ]:
plt.style.use("fivethirtyeight")
plt.rcParams['figure.figsize'] = 9, 9

One model per zip code works well if a zip code has many observations:

In [ ]:
sns.regplot(x=berlin["livingSpace"][berlin.geo_plz == "13583"], y=berlin["totalRent"][berlin.geo_plz == "13583"], 
            color="#42b883", ci=0, scatter_kws={"s": 40}, label="Spandau")
sns.regplot(x=berlin["livingSpace"][berlin.geo_plz == "10405"], y=berlin["totalRent"][berlin.geo_plz == "10405"], 
            color="#f07855",  ci=0, scatter_kws={"s": 40}, label="Prenzlauer Berg")

plt.scatter(berlin["livingSpace"], berlin["totalRent"], s=3, c="gray", alpha=0.3)
plt.ylabel("Monthly Rent [€]")
plt.xlabel("Living Area [sqm]")
plt.legend()
plt.show()

If a zip code has only a handful or less observations, it is a bit more difficult. In this zip code, we only have two observations and a model fit on these two points would result in a negative slope which doesn't make much sense. In cases where we have little data, we would prefer the model to be closer to a model fit on all data, as we did before. Since even in Blankenburg, in general, bigger flats should be more expensive.

In [ ]:
plt.scatter(berlin["livingSpace"], berlin["totalRent"], s=3, c="gray", alpha=0.3)
plt.scatter(berlin["livingSpace"][berlin.geo_plz == "13129"], 
            berlin["totalRent"][berlin.geo_plz == "13129"], s=40, label="Blankenburg")
plt.legend()
plt.show()

We can achieve this by saying that for each linear model (one for each zip code), the parameters determining the slope and intercept of this model come from a common distribution. So all slope parameters for example would come from a Normal distribution centered around some value close to 4.4 (the slope parameter we obtained in our last model) but the slope for Prenzlauer Berg would be higher than average while Spandau would be lower than average and places like Blankenburg would stay close to the mean (and also have higher uncertainty).

Before, our model looked like this:

$$\begin{align*}
\text{rent} &\sim \text{Normal}(\mu, \sigma) \\
\mu &= \alpha + \beta \text{area} \\
\\
\alpha &\sim \text{Normal}(0, 10) \\
\beta &\sim \text{Normal}(0, 5) \\
\\
\sigma &\sim \text{HalfNormal}(5) 
\end{align*}$$

We know extend this as follows:
$$\begin{align*}
\text{rent} &\sim \text{Normal}(\mu, \sigma) \\
\mu &= \alpha_{[\text{ZIP}]} + \beta_{[\text{ZIP}]} \text{area} \\
\\
\alpha_{[\text{ZIP}]} &\sim \text{Normal}(\mu_{\alpha}, \sigma_{\alpha}) \\
\beta_{[\text{ZIP}]} &\sim \text{Normal}(\mu_{\beta}, \sigma_{\beta}) \\
\\
\mu_{\alpha} &\sim \text{Normal}(0, 10) \\
\mu_{\beta} &\sim \text{Normal}(0, 5) \\
\\
\sigma, \sigma_{\alpha}, \sigma_{\beta} &\sim \text{HalfNormal}(5) 
\end{align*}$$

This looks like an awful lot more formula but the most important changes are only in the second, third and fourth line, the lines below are mostly repeating priors from above.

In the second line, we use a linear model similar to above, but with different $\alpha$ and $\beta$ for each ZIP code. As before, we need to define priors for these two parameters. Only unlike above, we now put so called hyperpriors on the parameters of these priors. $\mu_{\alpha}$ is now the expected mean for the intercepts of each ZIP code and $\sigma_{\alpha}$ determines how much of a difference there can be between the intercept in Spandau and the intercept in Prenzlauer Berg.

Don't worry if so many formulas are not your cup of tea, we now gonna look at the code for the model. Before writing the model however, we need to map the zip codes to an index variable:

In [ ]:
berlin["zip"] = berlin.geo_plz.map(str.strip)
zip_codes = np.sort(berlin.zip.unique())
num_zip_codes = len(zip_codes)
zip_lookup = dict(zip(zip_codes, range(num_zip_codes)))
berlin["zip_code"] = berlin.zip.replace(zip_lookup).values

And a small helper function to map from ZIP string to ZIP code:

In [ ]:
def map_zip_codes(zip_strings, zip_lookup=zip_lookup):
    return pd.Series(zip_strings).replace(zip_lookup).values

In [ ]:
with pm.Model() as hier_model:
    mu_alpha = ...
    sigma_alpha = ...
    
    mu_beta = ...
    sigma_beta = ...
    
    alpha = ...
    
    beta = ...
    

    sigma = ...
    
    mu = ...
    
    rent = pm.Normal("rent", mu=mu, sd=sigma, observed=berlin["totalRent_s"])
    
    trace = pm.sample(random_seed=2020, chains=4, 
                      draws=1000, tune=1000)

We will need to do the convergency checks again. For this, we first collect the model artifacts in an ArviZ Data object:

In [ ]:
import arviz as az
pm_data = ...

One thing where the ArviZ Data object is very useful, is for dealing with high-dimensional model artificats. With hierararchical models such as this one, we get one parameter per level (here the ZIP codes) and 4000 samples per parameter, coming from four chains (at least in the beginning, we want to keep the samples of each chain separated). ArviZ makes it easier to keep track of all the different dimensions and coordinates:

In [ ]:
pm_data.posterior

There are the dimensions `alpha_dim_0` and `beta_dim_0` that represent the ZIP codes. We have 208 ZIP codes and since we mapped them to an index, the coordinates for these dimensions are simply the integers from 0 to 207. But we can give them meaningful names by providing ArviZ with the original ZIP code strings:

In [ ]:
pm_data = az.from_pymc3(model=hier_model, trace=trace,
                        # create a new coordinate
                        ...
                        # this coordinate is used by the dimension alpha and beta
                       ...
                       )

In [ ]:
pm_data.posterior

There's much more that can be done with the ArviZ data object but this would go beyond this tutorial. A small introduction on how to work with them can be found [here](https://github.com/corriebar/arviz/blob/xarray-example/doc/notebooks/Working%20with%20InferenceData.ipynb).


Next, we'll check the trace plots. Remember, ArviZ has a function for this. Only problem: we know have many many alpha and beta parameters, one each for each ZIP code. This is way too much to plot! Use the function parameter `var_names` to only select the parameters from the model that don't use ZIP code as index.

Next, we'll check the three different summary/diagnostic statistics. There were R_hat, MCSE (Monte Carlo Standard Error), and ESS (Effective Sample Size). You can get the summary table using ArviZ again:

Unfortunately, since we have so many parameters, we can't check them easily by hand. What we can do instead, is to plot a histogram for each of the diagnostics:

In [ ]:
# rhat
summ = az.summary(pm_data, round_to=5)

...
plt.title("R_hat")
plt.show()

Check for yourself the ESS diagnostics:

In [ ]:
# ess
...
plt.title("ESS Mean")
plt.show()

Before, we checked how good our model fit the data by comparing the plot of the linear model to our data. Since we now have a collection of linear models, this would be rather difficult. What we can do instead is a so called posterior-predictive check. We compare the predicted distribution of outcomes to the actual distribution of outcomes.

In [ ]:
with hier_model:
    posterior_pred = ...

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(14,14), sharex=True)
ax = ax.ravel()

...

ax[0].set_title("Original data")

sample_nums = np.random.choice(posterior_pred["rent"].shape[0], size=3, replace=False)
for i, samp in enumerate(sample_nums):
    
    ...
    ax[i+1].set_title(f"Sample {i+1}")
plt.suptitle("Comparing Original Data to Predicted Data")
plt.show()

Even though it is difficult to visualize all models, we can pick out a few and check how the model differs for different ZIP codes. To now be able to select the posterior sample of a single ZIP code, the ArviZ object becomes very helpful. If we, for example, want to extract the sample for Blankenfelde (the ZIP code 13129 from above with so few observations), we get the data as follows:

In [ ]:
blankenfelde = ... 


In [ ]:
area_s = np.linspace(start=-2, stop=3.5, num=50)

mu_pred_blankenfelde = ...


# destandardize area again
area = destandardize_area(area_s)

plt.plot(area, mu_pred_blankenfelde.mean(1)*100, alpha=0.3, c="k")
plt.scatter(berlin["livingSpace"], berlin["totalRent_s"]*100, s=4, alpha=0.4, c="grey")

plt.title("Uncertainty (mu) for Blankenburg")


az.plot_hpd(area, mu_pred_blankenfelde.T*100, credible_interval=0.83)

plt.scatter(berlin["livingSpace"][berlin.geo_plz == "13129"], 
            berlin["totalRent"][berlin.geo_plz == "13129"], s=40, label="Blankenburg")
plt.xlabel('Living Area [sqm]')
plt.ylabel('Rent [€]')
plt.legend()
plt.show()

We can compare this with one of the ZIP codes from above that had more data.
Go and make the same plot for a different ZIP code (of your choice)!

If you want, you can also check how the full uncertainty looks like for these ZIP codes. Remember, for this you'll need to compute the predictions for rent.

In [ ]:
import scipy.stats as stats

In [ ]:
rent_blankenfelde = ...

plt.plot(area, mu_pred_blankenfelde.mean(1)*100, alpha=0.3, c="k")
plt.scatter(berlin["livingSpace"], berlin["totalRent_s"]*100, s=4, alpha=0.7, c="grey")

az.plot_hpd(area, mu_pred_blankenfelde.T*100, credible_interval=0.83, 
            fill_kwargs={"alpha": 0.5})

az.plot_hpd(area, rent_blankenfelde.T*100, credible_interval=0.83, 
            fill_kwargs={"alpha": 0.5})
plt.scatter(berlin["livingSpace"][berlin.geo_plz == "13129"], 
            berlin["totalRent"][berlin.geo_plz == "13129"], s=40, label="Blankenburg")

plt.legend()
plt.title("Full uncertainty for Blankenfelde")
plt.xlabel('Living Area [sqm]')
plt.ylabel('Rent [€]')
plt.show()

In [ ]:
# repeat for ZIP code of your choice

Instead of computing the rent predictions by hand, we could also use PyMC data container to handle predictions on new data.

Unfortunately, because of an still unclosed issue, we can't use the PyMC data container to update the ZIP code indices but need to use a shared theano variable. However, both types are updated in a similar fashion.

In [ ]:
zips = ...

with pm.Model() as hier_model:
    
    area = ...
    
    mu_alpha = pm.Normal("mu_alpha", mu=0, sigma=10)
    sigma_alpha = pm.HalfNormal("sigma_alpha", sigma=5)
    
    mu_beta = pm.Normal("mu_beta", mu=0, sigma=5)
    sigma_beta = pm.HalfNormal("sigma_beta", sigma=5)
    
    alpha = pm.Normal("alpha", mu=mu_alpha, sd=sigma_alpha, 
                      shape=num_zip_codes)
    
    beta = pm.Normal("beta", mu=mu_beta, sd=sigma_beta, 
                     shape=num_zip_codes)
    

    sigma = pm.HalfNormal("sigma", sigma=5)
    
    mu = alpha[zips] + beta[zips]*area
    
    rent = pm.Normal("rent", mu=mu, sd=sigma, observed=berlin["totalRent_s"])
    
    trace = pm.sample(random_seed=2020, chains=4, 
                      draws=1000, tune=1000)

Feel free to change the area and ZIP code data to for example your own flat.

In [ ]:
more_flats = pd.DataFrame({"area": standardize_area(np.array([100, 240, 74])), 
                           "zip_code": ["10243", "10179", "12047"]})

more_flats["zip"] = map_zip_codes(more_flats["zip_code"])

In [ ]:
with hier_model:
    zips.set_value(more_flats["zip"])
    pm.set_data({"area": more_flats["area"]})
    post_pred = pm.sample_posterior_predictive(trace, samples=1000)

As before, we can now plot this as a histogram:

In [ ]:
y_pred = post_pred["rent"][:,2]*100

plt.hist(y_pred, ec="darkblue", alpha=0.9, bins=20)
plt.title("Rental price distribution\nfor a flat of 74sqm in 12047")
plt.show()

Or ask for the probability that your flat would have a rent lower than your own rent: